In [1]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import keras
import datetime
from tqdm import tqdm
from sklearn.model_selection import TimeSeriesSplit
from catboost import CatBoostRegressor
import matplotlib.pyplot as plt

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [2]:
#This notebooks is a work in progress and incomplete.

In [3]:
train=pd.read_csv("/kaggle/input/playground-series-s3e19/train.csv",index_col='id')
test=pd.read_csv("/kaggle/input/playground-series-s3e19/test.csv",index_col='id')

In [4]:
train.head()

,date,country,store,product,num_sold
id,,,,,
0,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Improve Your Coding,63
1,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Train More LLMs,66
2,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Win Friends and Influence People,9
3,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Win More Kaggle Competitions,59
4,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Write Better,49


In [5]:
len(train['date'].unique())

1826

In [6]:
cat_features=['country','store','product']
for feature in cat_features:
    print(feature,':',train[feature].unique())

country : ['Argentina' 'Canada' 'Estonia' 'Japan' 'Spain']
store : ['Kaggle Learn' 'Kaggle Store' 'Kagglazon']
product : ['Using LLMs to Improve Your Coding' 'Using LLMs to Train More LLMs'
 'Using LLMs to Win Friends and Influence People'
 'Using LLMs to Win More Kaggle Competitions' 'Using LLMs to Write Better']


In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136950 entries, 0 to 136949
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   date      136950 non-null  object
 1   country   136950 non-null  object
 2   store     136950 non-null  object
 3   product   136950 non-null  object
 4   num_sold  136950 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 6.3+ MB


In [8]:
def datetime_features(df_temp):
    """
ref:-https://www.kaggle.com/code/chingiznurzhanov/timeseriessplit-catboost-trick
    """
    df_temp['month'] = df_temp['date'].dt.month
    df_temp['day'] = df_temp['date'].dt.day
    df_temp['year'] = df_temp['date'].dt.year
    df_temp['dayofweek'] = df_temp['date'].dt.dayofweek
    df_temp['quarter'] = df_temp['date'].dt.quarter
    df_temp['dayofmonth'] = df_temp['date'].dt.day
    #df_temp['weekofyear'] = df_temp['date'].dt.weekofyear
    df_temp['weekofyear'] = df_temp['date'].dt.isocalendar().week.astype(np.int64)
    df_temp['is_month_start'] = df_temp.date.dt.is_month_start.astype(np.int8)
    df_temp['is_month_end'] = df_temp.date.dt.is_month_end.astype(np.int8)
    df_temp['monday'] = df_temp.date.dt.weekday.eq(0).astype(np.uint8)
    df_temp['tuesday'] = df_temp.date.dt.weekday.eq(1).astype(np.uint8)
    df_temp['wednesday'] = df_temp.date.dt.weekday.eq(2).astype(np.uint8)
    df_temp['thursday'] = df_temp.date.dt.weekday.eq(3).astype(np.uint8)
    df_temp['friday'] = df_temp.date.dt.weekday.eq(4).astype(np.uint8)
    df_temp['saturday'] = df_temp.date.dt.weekday.eq(5).astype(np.uint8)
    df_temp['sunday'] = df_temp.date.dt.weekday.eq(6).astype(np.uint8)
    df_temp['is_quarter_end'] = df_temp['date'].dt.is_quarter_end.astype(np.uint8)
    df_temp['is_quarter_start'] = df_temp['date'].dt.is_quarter_start.astype(np.uint8)
    df_temp['is_year_end'] = df_temp['date'].dt.is_year_end.astype(np.uint8)
    df_temp['is_year_start'] = df_temp['date'].dt.is_year_start.astype(np.uint8)
    df_temp=df_temp.drop('date',axis=1)
    return df_temp

In [9]:
def data_encoding(dataset):
    cat_features=['country','store','product']
    for feature in tqdm(cat_features):
        dictionary={}
        n=1
        print('feature:',feature)
        print('categories:',dataset[feature].unique())
        for sub_cat in dataset[feature].unique():
            dictionary[sub_cat]=n
            n+=1
        print('encoding rule:',dictionary)
        dataset[feature]=dataset[feature].map(dictionary)
        dummies=pd.get_dummies(dataset[feature],prefix=feature)
        for key in dummies:
            dataset[key]=dummies[key]
        dataset=dataset.drop(feature,axis=1)
    dataset['date']=pd.to_datetime(dataset['date'],format='%Y-%m-%d')
    print("Date converted to Datetime features. Check returned dataframe if you want further details.")
    return datetime_features(dataset)

In [10]:
train=data_encoding(train)
test=data_encoding(test)

  0%|          | 0/3 [00:00<?, ?it/s]

feature: country
categories: ['Argentina' 'Canada' 'Estonia' 'Japan' 'Spain']
encoding rule: {'Argentina': 1, 'Canada': 2, 'Estonia': 3, 'Japan': 4, 'Spain': 5}
feature: store
categories: ['Kaggle Learn' 'Kaggle Store' 'Kagglazon']
encoding rule: {'Kaggle Learn': 1, 'Kaggle Store': 2, 'Kagglazon': 3}


100%|██████████| 3/3 [00:00<00:00, 14.84it/s]

feature: product
categories: ['Using LLMs to Improve Your Coding' 'Using LLMs to Train More LLMs'
 'Using LLMs to Win Friends and Influence People'
 'Using LLMs to Win More Kaggle Competitions' 'Using LLMs to Write Better']
encoding rule: {'Using LLMs to Improve Your Coding': 1, 'Using LLMs to Train More LLMs': 2, 'Using LLMs to Win Friends and Influence People': 3, 'Using LLMs to Win More Kaggle Competitions': 4, 'Using LLMs to Write Better': 5}


Date converted to Datetime features. Check returned dataframe if you want further details.


100%|██████████| 3/3 [00:00<00:00, 56.06it/s]

feature: country
categories: ['Argentina' 'Canada' 'Estonia' 'Japan' 'Spain']
encoding rule: {'Argentina': 1, 'Canada': 2, 'Estonia': 3, 'Japan': 4, 'Spain': 5}
feature: store
categories: ['Kaggle Learn' 'Kaggle Store' 'Kagglazon']
encoding rule: {'Kaggle Learn': 1, 'Kaggle Store': 2, 'Kagglazon': 3}
feature: product
categories: ['Using LLMs to Improve Your Coding' 'Using LLMs to Train More LLMs'
 'Using LLMs to Win Friends and Influence People'
 'Using LLMs to Win More Kaggle Competitions' 'Using LLMs to Write Better']
encoding rule: {'Using LLMs to Improve Your Coding': 1, 'Using LLMs to Train More LLMs': 2, 'Using LLMs to Win Friends and Influence People': 3, 'Using LLMs to Win More Kaggle Competitions': 4, 'Using LLMs to Write Better': 5}
Date converted to Datetime features. Check returned dataframe if you want further details.


In [11]:
train.head()

,num_sold,country_1,country_2,country_3,country_4,country_5,store_1,store_2,store_3,product_1,...,tuesday,wednesday,thursday,friday,saturday,sunday,is_quarter_end,is_quarter_start,is_year_end,is_year_start
id,,,,,,,,,,,,,,,,,,,,,
0,63,1,0,0,0,0,1,0,0,1,...,0,0,0,0,0,1,0,1,0,1
1,66,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,1,0,1
2,9,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,1,0,1
3,59,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,1,0,1
4,49,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,1,0,1


In [12]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136950 entries, 0 to 136949
Data columns (total 34 columns):
 #   Column            Non-Null Count   Dtype
---  ------            --------------   -----
 0   num_sold          136950 non-null  int64
 1   country_1         136950 non-null  uint8
 2   country_2         136950 non-null  uint8
 3   country_3         136950 non-null  uint8
 4   country_4         136950 non-null  uint8
 5   country_5         136950 non-null  uint8
 6   store_1           136950 non-null  uint8
 7   store_2           136950 non-null  uint8
 8   store_3           136950 non-null  uint8
 9   product_1         136950 non-null  uint8
 10  product_2         136950 non-null  uint8
 11  product_3         136950 non-null  uint8
 12  product_4         136950 non-null  uint8
 13  product_5         136950 non-null  uint8
 14  month             136950 non-null  int64
 15  day               136950 non-null  int64
 16  year              136950 non-null  int64
 17  dayofweek 

In [13]:
X,y=train.drop("num_sold",axis=1),train["num_sold"]

In [14]:
#(tf.convert_to_tensor(X),tf.convert_to_tensor(y))

In [15]:
cb_reg=CatBoostRegressor(loss_function='MAPE',verbose=3)
cb_reg.fit(X,y)

0:	learn: 0.7681344	total: 82.7ms	remaining: 1m 22s
3:	learn: 0.7063613	total: 145ms	remaining: 36.1s
6:	learn: 0.6509865	total: 209ms	remaining: 29.6s
9:	learn: 0.6005779	total: 271ms	remaining: 26.8s
12:	learn: 0.5556023	total: 333ms	remaining: 25.3s
15:	learn: 0.5154378	total: 396ms	remaining: 24.3s
18:	learn: 0.4797088	total: 457ms	remaining: 23.6s
21:	learn: 0.4486643	total: 519ms	remaining: 23.1s
24:	learn: 0.4224866	total: 582ms	remaining: 22.7s
27:	learn: 0.4004820	total: 644ms	remaining: 22.4s
30:	learn: 0.3826975	total: 702ms	remaining: 21.9s
33:	learn: 0.3686199	total: 762ms	remaining: 21.6s
36:	learn: 0.3567323	total: 822ms	remaining: 21.4s
39:	learn: 0.3470778	total: 882ms	remaining: 21.2s
42:	learn: 0.3380456	total: 942ms	remaining: 21s
45:	learn: 0.3305483	total: 999ms	remaining: 20.7s
48:	learn: 0.3238798	total: 1.06s	remaining: 20.6s
51:	learn: 0.3174818	total: 1.12s	remaining: 20.4s
54:	learn: 0.3116046	total: 1.18s	remaining: 20.2s
57:	learn: 0.3059675	total: 1.23s	r

In [16]:
cb_reg.predict(test)

array([ 37.65109003,  35.47547464,   5.00696186, ...,  80.37452566,
       425.80887339, 361.95864796])

In [17]:
submission=pd.DataFrame(index=test.index)
#submission['id']=test.index
num_sold=[round(x) for x in cb_reg.predict(test)]
submission['num_sold']=num_sold
submission.to_csv("submission.csv",index='id')

In [18]:
submission

,num_sold
id,
136950,38
136951,35
136952,5
136953,30
136954,27
...,...
164320,525
164321,491
164322,80
